# Speeding up Python code

In this notebook we try out methods to speed up the execution of Python code.
Some of the possibilities are:
* [Memoization](#Memoization)
* [Numba](#Numba)
* [Cython](#Cython)
* [Parallelizing](#Parallelizing)

To assess the speed-ups we have various [Timing tools](#Timing_tools) and to identify places where the code takes 

Refs:

https://numba.readthedocs.io/en/stable/index.html

https://www.scaler.com/topics/python-parallel-for-loop/

In [10]:
import numpy as np

## Timing tools <a id='Timing_tools'></a>

Within a Jupyter notebook we can use `%timeit` in front of a single command (e.g., a function call) or `%%time` for a cell.

### Python timing tools

See https://realpython.com/python-timer/ for further details.

## Memoization <a id='Memoization'></a>

In [19]:
from math import sin, sqrt

def sin_half_plain(x):
    return sin(x)/2

In [2]:
from functools import lru_cache
from math import sin

@lru_cache(360)   # cache at most 360 values
def sin_half(x):
    return sin(x)/2


In [34]:
# from https://www.dice.com/career-advice/5-ways-to-make-your-python-programs-run-faster
from time import perf_counter_ns
from functools import lru_cache

def main():    
    def fib(n):
        return n if n < 2 else fib(n-1) + fib(n-2)

    @lru_cache(maxsize=None)
    def mfib(n):
        return n if n < 2 else mfib(n-1) + mfib(n-2)
    start=perf_counter_ns()
    print(f"Non-memoized fib()={fib(35)}")
    end=perf_counter_ns()      
    print(f"Time took {end-start}")

    start=perf_counter_ns()  
    print(f"Memoized fib()={mfib(35)}")
    end=perf_counter_ns()      
    print(f"Time took {end-start}")

main()

Non-memoized fib()=9227465
Time took 2306010792
Memoized fib()=9227465
Time took 49334


## Numba

From https://towardsdatascience.com/how-to-skyrocket-your-python-speed-with-numba-89fd16362e47

In [4]:
import random

def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [5]:
%timeit monte_carlo_pi(10000)

2.55 ms ± 30.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
from numba import jit # <-- importing jit from numba
import random

@jit(nopython=True) # <-- The only difference
def monte_carlo_pi_numba(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples


In [8]:
%timeit monte_carlo_pi_numba(10000)

75.8 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
import numpy
from numba import jit

def numpy_features(matrix: np.array) -> None:
    """Illustrates some common features of NumPy."""
    cosine_trace = 0.0
    for i in range(matrix.shape[0]):
        cosine_trace += np.cos(matrix[i, i])
    matrix = matrix + cosine_trace


In [12]:
x = np.arange(1000000).reshape(1000, 1000)
%time numpy_features(x)


CPU times: user 7.72 ms, sys: 4.94 ms, total: 12.7 ms
Wall time: 12 ms


In [13]:
import numpy
from numba import jit

@jit(nopython=True)
def numpy_features_numba(matrix: np.array) -> None:
    """Illustrates some common features of NumPy."""
    cosine_trace = 0.0
    for i in range(matrix.shape[0]):
        cosine_trace += np.cos(matrix[i, i])
    matrix = matrix + cosine_trace

In [16]:
x = np.arange(1000000).reshape(1000, 1000)
%time numpy_features_numba(x)


CPU times: user 213 ms, sys: 8.43 ms, total: 222 ms
Wall time: 233 ms


So slow!  But now run again.

In [17]:
x = np.arange(1000000).reshape(1000, 1000)
%time numpy_features(x)


CPU times: user 3.49 ms, sys: 911 µs, total: 4.4 ms
Wall time: 5.18 ms


Not such a large speed-up as found in the article.

From http://numba.pydata.org/numba-doc/dev/user/5minguide.html

In [36]:
from numba import jit
import numpy as np

x = np.arange(100).reshape(10, 10)

def go_fast_nonumba(a): 
    trace = 0.0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def go_fast_numba(a): # Function is compiled to machine code when called the first time
    trace = 0.0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

%timeit go_fast_nonumba(x)
%timeit go_fast_numba(x)
%timeit go_fast_numba(x)


13.9 µs ± 26.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
955 ns ± 7.01 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
958 ns ± 5.05 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### test_mandelbrot

In [ ]:
from timeit import default_timer as timer
try:
    from matplotlib.pylab import imshow, show
    have_mpl = True
except ImportError:
    have_mpl = False
import numpy as np
from numba import jit

@jit(nopython=True)
def mandel(x, y, max_iters):
    """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
    """
    i = 0
    c = complex(x,y)
    z = 0.0j
    for i in range(max_iters):
        z = z * z + c
        if (z.real * z.real + z.imag * z.imag) >= 4:
            return i

    return 255

@jit(nopython=True)
def create_fractal(min_x, max_x, min_y, max_y, image, iters):
    height = image.shape[0]
    width = image.shape[1]

    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height
    for x in range(width):
        real = min_x + x * pixel_size_x
        for y in range(height):
            imag = min_y + y * pixel_size_y
            color = mandel(real, imag, iters)
            image[y, x] = color

    return image

image = np.zeros((500 * 2, 750 * 2), dtype=np.uint8)
s = timer()
create_fractal(-2.0, 1.0, -1.0, 1.0, image, 20)
e = timer()
print(e - s)
if have_mpl:
    imshow(image)
    show()

From https://pythonspeed.com/articles/numba-faster-python/

In [44]:
a_test = np.ones(10000000)

In [45]:
def monotonically_increasing(a):
    max_value = 0
    for i in range(len(a)):
        if a[i] > max_value:
            max_value = a[i]
        a[i] = max_value

In [46]:
from numba import njit

@njit
def monotonically_increasing_numba(a):
    max_value = 0
    for i in range(len(a)):
        if a[i] > max_value:
            max_value = a[i]
        a[i] = max_value

In [47]:
%timeit monotonically_increasing(a_test)

1.38 s ± 30.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
%timeit monotonically_increasing_numba(a_test)

6.25 ms ± 20 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Random speed-ups

From https://hungyuling.com/blog/make-python-run-faster/

Try with numba

In [18]:
%timeit 3.1415 ** 0.5

8.47 ns ± 0.0338 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [20]:
%timeit sqrt(3.1415)

43.7 ns ± 0.043 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [21]:
%timeit np.sqrt(3.1415)

729 ns ± 2.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [22]:
a = np.array(3.1415)
%timeit np.sqrt(a)

518 ns ± 1.13 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Be more pythonic

In [28]:
from time import perf_counter_ns

# non pythonic
start = perf_counter_ns()
total = 0 
for i in range(1,100):
    if (i %3) == 0:
        total += i
end=perf_counter_ns()      
print (f"Non-Pythonic Total of divisible by 3= {total}")
print(f"Time took {end-start}")

# pythonic
start = perf_counter_ns()
total = sum(range(1, 100, 3))
end=perf_counter_ns()      
print (f"Pythonic Total of divisible by 3= {total}")
print(f"Time took {end-start}")



Non-Pythonic Total of divisible by 3= 1683
Time took 259500
Pythonic Total of divisible by 3= 1617
Time took 119583


In [31]:
from time import perf_counter_ns

def main():
    # non pythonic
    start = perf_counter_ns()
    total = 0 
    for i in range(1,100):
        if (i %3) == 0:
            total += i
    end=perf_counter_ns()      
    print (f"Non-Pythonic Total of divisible by 3= {total}")
    print(f"Time took {end-start}")
    
    # pythonic
    start = perf_counter_ns()
    total = sum(range(1, 100, 3))
    end=perf_counter_ns()      
    print (f"Pythonic Total of divisible by 3= {total}")
    print(f"Time took {end-start}")

main()

Non-Pythonic Total of divisible by 3= 1683
Time took 25250
Pythonic Total of divisible by 3= 1617
Time took 4917


In [26]:
%%time
total = 0 
for i in range(1,100):
    if (i %3) == 0:
        total += i


CPU times: user 46 µs, sys: 26 µs, total: 72 µs
Wall time: 143 µs


In [27]:
%%time
total = sum(range(1, 100, 3))


CPU times: user 13 µs, sys: 19 µs, total: 32 µs
Wall time: 46.7 µs


## Multiprocessing

### Method1: Use the Multiprocessing Module

`multiprocessing.Pool().map()` is a good choice for parallelizing simple loops. To parallelize the loop the multiprocessing package provides a process pool with helpful functions to automatically manage a pool of worker processes. By default, the created Pool class instance uses all available CPU cores. The parallel version of the built-in map() function takes a single argument. It calls the same function for every data in the provided iterable and returns an iterable of return values from the target function. Once all the work is done using the close() function the pool_obj is closed to release the worker processes.

In [33]:
import multiprocessing
def printVal(value):
    return ("hello "*(value))
pool_obj = multiprocessing.Pool()
ans = pool_obj.map(printVal,range(0,4))
print(ans)
pool_obj.close()

Process SpawnPoolWorker-4:
Process SpawnPoolWorker-2:
Process SpawnPoolWorker-1:
Process SpawnPoolWorker-3:
Traceback (most recent call last):
  File "/Users/furnstah/Dropbox/Anaconda/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/furnstah/Dropbox/Anaconda/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/furnstah/Dropbox/Anaconda/anaconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/furnstah/Dropbox/Anaconda/anaconda3/lib/python3.9/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
  File "/Users/furnstah/Dropbox/Anaconda/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
AttributeError: Can't get attribute 'printVal' on <module '__main__' (built-in)>
Traceba

KeyboardInterrupt: 

### Method 2: Use the Joblib Module

Joblib is a set of tools to provide lightweight pipelining in Python. It mainly aims to avoid computing the same thing twice. To perform parallel processing, we have to set the number of jobs, which is limited to the number of cores in the CPU. The delayed() function delays the mentioned method call for some time. The Parallel() function creates a parallel instance with the specified number of cores.

In [1]:
import joblib
import math
def multiple_of_six(i):
    return (i*6)
print(joblib.Parallel(n_jobs=2)(joblib.delayed(multiple_of_six)(i) for i in range(5)))

[0, 6, 12, 18, 24]


### Method 3: Use the Asyncio Module

The asyncio module is single-threaded that runs loops by suspending the sequence temporarily using await methods. The main function and called functions run parallelly without affecting each other. The loop also runs in parallel with the main function.

In [2]:
import asyncio
import time
def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)
    return wrapped
@background
def f(argument):
    time.sleep(2)
    print('Function', str(argument), 'Executed')
for i in range(5):
    f(i)
print('Loop finished before functions')

Loop finished before functions
Function 0 Executed
Function 1 Executed
Function 2 Executed
Function 3 Executed
Function 4 Executed


## memoization

https://www.dice.com/career-advice/5-ways-to-make-your-python-programs-run-faster



In [ ]:
from time import perf_counter_ns
from functools import lru_cache

def main():    
    def fib(n):
        return n if n < 2 else fib(n-1) + fib(n-2)

    @lru_cache(maxsize=None)
    def mfib(n):
        return n if n < 2 else mfib(n-1) + mfib(n-2)
    start=perf_counter_ns()
    print(f"Non-memoized fib()={fib(35)}")
    end=perf_counter_ns()      
    print(f"Time took {end-start}")

    start=perf_counter_ns()  
    print(f"Memoized fib()={mfib(35)}")
    end=perf_counter_ns()      
    print(f"Time took {end-start}")

# if __name__ == "__main__":
#     main()


## GPUs

In [52]:
from numba import jit, cuda
import numpy as np
# to measure exec time
from timeit import default_timer as timer

# normal function to run on cpu
def func(a):
    for i in range(10000000):
        a[i]+= 1

# function optimized to run on gpu
@jit(target_backend='cuda')
def func2(a):
    for i in range(10000000):
        a[i]+= 1
        
if __name__=="__main__":
    n = 10000000
    a = np.ones(n, dtype = np.float64)
    
    start = timer()
    func(a)
    print("without GPU:", timer()-start)
    
    start = timer()
    func2(a)
    print("with GPU:", timer()-start)


without GPU: 1.7374667909462005
with GPU: 0.03767174994572997


In [51]:
from numba import jit, cuda
import numpy as np
# to measure exec time
from timeit import default_timer as timer

# normal function to run on cpu
def func(a):
    for i in range(10000000):
        a[i]+= 1

# function optimized to run on gpu
@jit(target_backend='cuda')
def func2(a):
    for i in range(10000000):
        a[i]+= 1
        
if __name__=="__main__":
    n = 10000000
    a = np.ones(n, dtype = np.float64)
    
    start = timer()
    func(a)
    print("without GPU:", timer()-start)
    
    start = timer()
    func2(a)
    print("with GPU:", timer()-start)


without GPU: 1.783250084030442
with GPU: 0.03734833304770291


In [53]:
%timeit func(a)

1.68 s ± 4.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
%timeit func2(a)

1.84 ms ± 7.49 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
